In [2]:
import numpy as np 
import matplotlib.pyplot as plt

# install
## numpy
## matplotlib

## data load & preprocessing

In [3]:
from dataset.mnist import load_mnist

(train_raw_img, train_label), (test_raw_img, test_label) = load_mnist(flatten=False, normalize=False)
print(train_raw_img.shape)

(60000, 1, 28, 28)


In [4]:
# preprocessing (train & inference)

train_img = train_raw_img.reshape(len(train_raw_img.squeeze()), -1)
train_label = train_label.reshape(len(train_label), -1)

test_img = test_raw_img.reshape(len(test_raw_img.squeeze()), -1)
test_label = test_label.reshape(len(test_label), -1)

print(train_img.shape)
print(train_label.shape)
print(test_img.shape)
print(test_label.shape)

(60000, 784)
(60000, 1)
(10000, 784)
(10000, 1)


In [5]:
# normalization (set value 0 ~ 1)

train_img = train_img.astype('float')
train_img = train_img/255

test_img = test_img.astype('float')
test_img = test_img/255

## model

In [ ]:
class Linear :
    def __init__(self, input_size=1, hidden_size=1) :
        self.W = 
        self.b = 
        self.x = 
        self.dW = 
        self.db = 
    
    def forward(self, x) :
        pass
    
    def backward(self, dout, lr) :
        pass

In [ ]:
class Relu :
    def __init__(self) :
        self.mask = None
    
    def forward(self, x) :
        self.mask = (x < 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout) :
        dout[self.mask] = 0
        return dout                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
class softmax_with_crossEntropy :
    def __init__(self) :
        self.delta = 1e-7
        # softmax
        self.softmax_x = 
        self.softmax_out = 
        # crossEntropy
        self.pred = 
        self.target = 
        
    def softmax_forward(self, x) :
        pass
    
    def crossEntropy_forward(self, pred, target) :
        pass
    
    def backward(self) :
        pass

## Train

### base

In [ ]:
# one_hot label 만드는 함수

def make_one_hot(labels) :
    a = []
    for label in labels :
        one_hot = np.zeros(10)
        one_hot[label] = 1
        a.append(one_hot)
    a = np.array(a)
    return a

# one_hot_labels = make_one_hot(train_label)
# print(train_label[0])
# print(one_hot_labels[0])

In [ ]:
# train version 1

from collections import OrderedDict

def train_MLP(config) :
    lr, num_epoch = config['learning_rate'], config['num_epoch']
    print_loss_interval = 1
    
    layer1 = Linear(784, 100)
    relu = Relu()
    layer2 = Linear(100, 10)
    softmax_with_CE = softmax_with_crossEntropy()
    
    for iter in range(num_epoch) :
        # forward
        x = layer1.forward(train_img)
        x = relu.forward(x)
        x = layer2.forward(x)
        preds = softmax_with_CE.softmax_forward(x)
        
        # loss
        one_hot_labels = make_one_hot(train_label)
        losses = softmax_with_CE.crossEntropy_forward(preds, one_hot_labels)
        loss = losses.sum()/len(preds)
        
        if iter % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter, num_epoch, loss))
        
        # backward
        dL = softmax_with_CE.backward()
        dL = layer2.backward(dL, lr)
        dL = relu.backward(dL)
        dL = layer1.backward(dL, lr)
            
    model = OrderedDict()
    model['layer1'] = layer1
    model['relu'] = relu
    model['layer2'] = layer2
    model['softmax_with_CE'] = softmax_with_CE
    
    return model

In [ ]:
config = { 'learning_rate' : 0.1,
            'num_epoch' : 100
          }

model = train_MLP(config)

In [ ]:
def eval(model, train_version = True) :
    if train_version :
        x = train_img
        labels = train_label.squeeze()
        print('In train dataset ... ')
    else : 
        x = test_img
        labels = test_label.squeeze()
        print('\nIn test dataset ... ')
    
    for layer in model.values() :
        if isinstance(layer, softmax_with_crossEntropy) :
            x = layer.softmax_forward(x)
        else :
            x = layer.forward(x)
            
    preds = x.argmax(axis=1)
    acc = np.sum(np.where(preds==labels, True, False))/len(labels)
    return acc

print('\t Accuracy :', eval(model, train_version=True))
print('\t Accuracy :', eval(model, train_version=False))
print()

### regularization

In [ ]:
class L2_regularization :
    pass

In [ ]:
def train(config) :
    
    return model

In [ ]:
config = { 'model' : MLP_Classifier(),
            'learning_rate' : 0.1,
            'num_epoch' : 100
          }

model = train(config)

In [ ]:
print('\t Accuracy :', eval(model, train_version=True))
print('\t Accuracy :', eval(model, train_version=False))
print()